# EVENT SIMULATION 
# Estudiante: Shirley Carminia Eguivar Villca

### Definicion del Problema:

 Este taller de costura  tiene  3  operarios(costureros) cada operario tiene una maquina para la confeccion de las prendas
 Pero solo existe una sola maquina que coloca los botones
 Los operarios tienen que  turnarse para acceder a la maquina de colocado de botones una vez que terminan con la confeccion de la prenda.
 
 * El colocado de botones tarda entre 10 a 25 minutos por  operario 
 * un operario solicita el uso de la maquina coloca botones cada 20 minutos  mas o menos
 
 Cuanto tiempo van a tardar en que todos los operarios accedan a la maquina coloca botones para que las prendas queden listas?


### #1 Importamos las librerias necesarias

In [199]:
import random
import math
import simpy

### #2 Constantes y variables

In [200]:

semilla = 15
nro_maquinas = 1
tiempo_min = 10
tiempo_max = 25
T_LLEGADAS = 20
#TIEMPO_SIMULACION = 120
nro_operarios = 6  # esa vez vamos a usar cantidad de operarios
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # Total del tiempo inverido en el colocado de botones
fin = 0.0 # minuto en el que finaliza todo

### #3 Procedimientos

In [205]:
# El procedimiento costurar_botones simula el servidor 
# Este procedimiento simula el uso de la maquina por el operario
# Y el tiempo que que tarda en el costurado de botones 
# Se utiliza la formula para el calculo de tiempo, considerar que el tiempo debe entrar dentro el rango de min a max
# tiempo_costurado = tiempo_min + (tiempo_max – tiempo_min) R 
# donde R es un nro aleatorio el resultado debe darnos ujn valor en el rando definido

def costurar_botones(operario):
    global dt  
    R = random.random()  
    tiempo = tiempo_max - tiempo_min  
    tiempo_costurado = tiempo_min + (tiempo*R) 
    yield env.timeout(tiempo_costurado) 
    #print('%7.4f %s: Botones listos' % (tiempo_costurado, operario))
    dt = dt + tiempo_costurado 


# El procedimiento simula a los operarios que acceden/solicitan  la maquina de colocado de botones
# Simula la llegada del operario, la salida

def operario (env, name, maquina_botones ):
    global te
    global fin    
    llega = env.now # Guarda el minuto de llegada del operario
    print('%7.4f %s: Solicitando usar la maquina de botones' % (llega, name))
    with maquina_botones.request() as request: 
        # Accede a la maquina
        yield request 
        # minuto que accede al servidor
        pasa = env.now 
        # tiempo que espera
        espera = pasa - llega 
        # Acumulador de tiempos de espera
        te = te + espera 
        print('%7.4f %s: Accede al uso de la maquina luego de esperar %6.4f min.' % (pasa, name, espera))
        yield env.process(costurar_botones(name)) # Invoca al proceso costurar_botones
        deja = env.now #Guarda el minuto en que termina el proceso costurar_botones 
        tiempo_costura =  deja-pasa
        print('%7.4f %s: Finaliza el colocado de botones luego de  %6.4f  min.' % (deja, name,tiempo_costura))
        fin = deja 

# Este procedimiento principal, que llama al operario y solicita el uso de la maquina coloca botones
def main (env, maquina_botones):
	llegada = 0
	i = 0
	for i in range(nro_operarios): # Para n operarios
		R = random.random()
        # Distribucion exponencial
		llegada = -T_LLEGADAS * math.log(R) 
		yield env.timeout(llegada)  
		i += 1
		env.process(operario(env, 'Operario %d' % i, maquina_botones))

### #4 Simulacion del Proceso

In [206]:
# Esta parte de codigo es la simulacion misma de "teoria de colas"
random.seed (semilla)  
env = simpy.Environment() 
# crea la maquina coloca botones (servidor)
maquina_botones = simpy.Resource(env, nro_maquinas) 
# llama al procediminto principal
env.process(main(env, maquina_botones)) 
# ejecuta la simulacion
env.run() 


 0.7075 Operario 1: Solicitando usar la maquina de botones
 0.7075 Operario 1: Accede al uso de la maquina luego de esperar 0.0000 min.
21.7474 Operario 1: Finaliza el colocado de botones luego de  21.0399  min.
89.7485 Operario 2: Solicitando usar la maquina de botones
89.7485 Operario 2: Accede al uso de la maquina luego de esperar 0.0000 min.
114.5435 Operario 2: Finaliza el colocado de botones luego de  24.7951  min.
126.6500 Operario 3: Solicitando usar la maquina de botones
126.6500 Operario 3: Accede al uso de la maquina luego de esperar 0.0000 min.
149.8424 Operario 3: Finaliza el colocado de botones luego de  23.1924  min.
208.2818 Operario 4: Solicitando usar la maquina de botones
208.2818 Operario 4: Accede al uso de la maquina luego de esperar 0.0000 min.
215.9553 Operario 5: Solicitando usar la maquina de botones
215.9590 Operario 6: Solicitando usar la maquina de botones
231.1419 Operario 4: Finaliza el colocado de botones luego de  22.8601  min.
231.1419 Operario 5: Acce

In [221]:
print ("\n---------------------------------------------------------------------")
print ("Tiempo total para finalizar el colocado de botones = %.2f" % fin)
print ("Nro de operarios: %.f" % nro_operarios)
print ("Nro de maquinas coloca botones: %.f" % nro_maquinas)
print ("\n---------------------------------------------------------------------")



---------------------------------------------------------------------
Tiempo total para finalizar el colocado de botones = 259.81
Nro de operarios: 6
Nro de maquinas coloca botones: 1

---------------------------------------------------------------------
